In [0]:
incremental_orders=spark.read.format("csv").option("header",True).option("inferSchema",True).load("dbfs:/FileStore/tables/orders/olist_orders_dataset.csv")

In [0]:
from pyspark.sql.functions import year, month, col
from delta.tables import DeltaTable

# Define the correct Delta Table path
target_table_path = "dbfs:/FileStore/tables/stream_write/"

# Extract year and month dynamically for partitioning
incremental_orders= incremental_orders.withColumn("order_year", year(col("order_purchase_timestamp"))) \
                                       .withColumn("order_month", month(col("order_purchase_timestamp")))

incremental_orders.write.format("csv") \
        .option("header", "true")\
        .mode("overwrite") \
        .partitionBy("order_year", "order_month") \
        .save(target_table_path)



In [0]:
dbutils.fs.rm("dbfs:/FileStore/tables/stream_write/",True)

In [0]:
df=spark.read.format("csv").option("header",True).load("dbfs:/FileStore/tables/stream_write/")

In [0]:
display(df)